In [ ]:
from collections import deque

#function that sets-up a nested dictionary where the resistor name is the key for the outer dictionary 
#and the information on nodes and resistance are key-value pairs as shown below
def dictResistors(resList, r_info):
    key = r_info[0]
    resList[key] = {
        "V1" : r_info[1],
        "V2" : r_info[2],
        "resistance" : int(r_info[3])
    }

#function that sets-up the resistor network as a graph using dictionary
#adjacency list using dictionary
#stores parallel connections as appended rname:rvalue pair in the inner dictionary
#reference: ChatGPT
def add_edge(adjacency_list, rname, u, v, rvalue):
    if u in adjacency_list:
        if v in adjacency_list[u]:
            adjacency_list[u][v].append({rname: rvalue}) 
        else:
            adjacency_list[u][v] = [{rname: rvalue}]
    else:
        adjacency_list[u] = {
            v: [{rname: rvalue}]
        }
    
    if v in adjacency_list:
        if u in adjacency_list[v]:
            adjacency_list[v][u].append({rname: rvalue}) 
        else:
            adjacency_list[v][u] = [{rname: rvalue}]
    else:
        adjacency_list[v] = {
            u: [{rname: rvalue}]
    }
        
#function that finds the shortest path using BFS from a start vertex to end vertex
#reference: ChatGPT
def shortest_path(graph, start_vertex, end_vertex):
    if start_vertex == end_vertex:
        return [start_vertex]

    visited = set()
    visited.add(start_vertex)

    queue = deque([(start_vertex, [])])

    while queue:
        current_vertex, path = queue.popleft()

        if current_vertex == end_vertex:
            return path + [current_vertex]

        for neighbor in graph[current_vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [current_vertex]))
    return None

#function that checks for parallel connection between two resistors
def check_parallel(res1, res2):
    vertices_res1 = [resList[res1]["V1"], resList[res1]["V2"]]
    vertices_res2 = [resList[res2]["V1"], resList[res2]["V2"]]
    if(set(vertices_res1) == set(vertices_res2)):
        return True
    else:
        return False
    
#function that checks for series connection between two resistors
def check_series(res1, res2):
    vertices_res1 = [resList[res1]["V1"], resList[res1]["V2"]]
    vertices_res2 = [resList[res2]["V1"], resList[res2]["V2"]]
    series = 1
    if vertices_res1 == ['Vdd', 'GND'] or vertices_res2 == ['Vdd', 'GND']:
        series = 0
    else:
        path = shortest_path(resGraph, vertices_res1[1],  vertices_res2[0])
        for i in path:
            if len(path)==1 and (i=="Vdd" or i=="GND"):
                series = 0
                break
            if i != path[0] and (i == "Vdd" or i == "GND"):
                series = 0
                break
            else:
                if len(resGraph[i].keys()) > 2:
                    series = 0
                    break

                for neighbors in resGraph[i]:
                    if len(resGraph[i][neighbors]) >= 2:
                        series = 0
                        break
    return series
        
N = int(input())

#transforms the input into a list of resistors called 'resList' by calling def dictResistors
resList = {}
for i in range(N):
    r_info = input().split()
    dictResistors(resList, r_info)
    
#transforms the data in the resList into a graph called 'resGraph' by calling def add_edge
resGraph = {}
for key in resList:
    add_edge(resGraph, key, resList[key]["V1"], resList[key]["V2"], resList[key]["resistance"])
    
#initializing variables 
R = []                 #stores the name of all resistors in the network
Rmarker = []           #also stores the name. purpose is to mark resistors that have connection
parallel_total = 0     #stores the equivalent parallel resistance
series_total = 0       #stores the equivalent series resistance
res_parallel = []      #stores resistors in parallel
                       #will manifest as a nested list if there are many sets of parallel resistors
res_series = []        #stores resistors in series
                       #will manifest as a nested list if there are many sets of series resistors

#loop that stores all the resistor names in R and Rmarker
for resistors in resList: #iterates through all the keys in the resList dictionary where the keys are the resistor names
    R.append(resistors)
    Rmarker.append(resistors)
    
for i in range(len(R)):
    res1 = R[i]
    if res1 not in Rmarker:
            continue
    else:
        for j in range(i+1, len(R)):
            res2 = R[j]
            if res2 not in Rmarker:
                continue
            else:
                if check_parallel(res1, res2):
                    present = 0
                    for i in range(len(res_parallel)):
                        if res1 in res_parallel[i] and res2 in Rmarker:
                            res_parallel[i].append(res2)
                            Rmarker.remove(res2)
                            present = 1
                            break
                    if present == 0:
                        res_parallel.append([res1,res2])
                        Rmarker.remove(res1)
                        Rmarker.remove(res2)

                elif check_series(res1, res2):
                    present = 0
                    for i in range(len(res_series)):
                        if res1 in res_series[i] and res2 in Rmarker:
                            res_series[i].append(res2)
                            Rmarker.remove(res2)
                            present = 1
                            break
                    if present == 0:
                        res_series.append([res1,res2])
                        Rmarker.remove(res1)
                        Rmarker.remove(res2)


for i in range(len(res_series)):
    for k in res_series[i]:
        series_total += resList[k]['resistance']
    if series_total != 0:
        print("[" + ', '.join(res_series[i]) + "]", series_total) 
        series_total = 0
        
for i in range(len(res_parallel)):
    for k in res_parallel[i]:
        parallel_total += 1/resList[k]['resistance']
    if parallel_total != 0:
        print("[" + ', '.join(res_parallel[i]) + "]", int(1/parallel_total)) 
        parallel_total = 0
        
            



    
    